In [1]:
!pip3 install pyspark

Defaulting to user installation because normal site-packages is not writeable


In [2]:
!pip3 install findspark

Defaulting to user installation because normal site-packages is not writeable


In [1]:
import findspark
findspark.init('/home/ubuntu/spark-3.3.1-bin-hadoop3')
findspark.find()

'/home/ubuntu/spark-3.3.1-bin-hadoop3'

In [2]:
from pyspark.sql import SparkSession

# The entry point into all functionality in Spark is the SparkSession class.
spark = (SparkSession
	.builder
	.appName("DS5110/CS5501: my awesome Spark program - balanced att1")
	.master("spark://172.31.8.146:7077")
	.config("spark.executor.memory", "1024M")
	.getOrCreate())

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/04/29 23:05:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

from pyspark.sql.types import *
from pyspark.sql.functions import col, count, desc, rand, when

from pyspark.ml.feature import ChiSqSelector
from pyspark.ml.stat import Correlation
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler, StringIndexer, StandardScaler, PCA
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

from pyspark.sql import functions as F


## specify the data type of each column 

In [5]:
# Define the schema for the DataFrame
schema = StructType([
    StructField('_c0', IntegerType()),
    StructField('Flow ID', StringType()),
    StructField('Src IP', StringType()),
    StructField('Src Port', IntegerType()),
    StructField('Dst IP', StringType()),
    StructField('Dst Port', IntegerType()),
    StructField('Protocol', IntegerType()),
    StructField('Timestamp', StringType()),
    StructField('Flow Duration', IntegerType()),
    StructField('Tot Fwd Pkts', IntegerType()),
    StructField('Tot Bwd Pkts', IntegerType()),
    StructField('TotLen Fwd Pkts', FloatType()),
    StructField('TotLen Bwd Pkts', FloatType()),
    StructField('Fwd Pkt Len Max', FloatType()),
    StructField('Fwd Pkt Len Min', FloatType()),
    StructField('Fwd Pkt Len Mean', FloatType()),
    StructField('Fwd Pkt Len Std', FloatType()),
    StructField('Bwd Pkt Len Max', FloatType()),
    StructField('Bwd Pkt Len Min', FloatType()),
    StructField('Bwd Pkt Len Mean', FloatType()),
    StructField('Bwd Pkt Len Std', FloatType()),
    StructField('Flow Byts/s', FloatType()),
    StructField('Flow Pkts/s', FloatType()),
    StructField('Flow IAT Mean', FloatType()),
    StructField('Flow IAT Std', FloatType()),
    StructField('Flow IAT Max', FloatType()),
    StructField('Flow IAT Min', FloatType()),
    StructField('Fwd IAT Tot', FloatType()),
    StructField('Fwd IAT Mean', FloatType()),
    StructField('Fwd IAT Std', FloatType()),
    StructField('Fwd IAT Max', FloatType()),
    StructField('Fwd IAT Min', FloatType()),
    StructField('Bwd IAT Tot', FloatType()),
    StructField('Bwd IAT Mean', FloatType()),
    StructField('Bwd IAT Std', FloatType()),
    StructField('Bwd IAT Max', FloatType()),
    StructField('Bwd IAT Min', FloatType()),
    StructField('Fwd PSH Flags', IntegerType()),
    StructField('Bwd PSH Flags', IntegerType()),
    StructField('Fwd URG Flags', IntegerType()),
    StructField('Bwd URG Flags', IntegerType()),
    StructField('Fwd Header Len', IntegerType()),
    StructField('Bwd Header Len', IntegerType()),
    StructField('Fwd Pkts/s', FloatType()),
    StructField('Bwd Pkts/s', FloatType()),
    StructField('Pkt Len Min', FloatType()),
    StructField('Pkt Len Max', FloatType()),
    StructField('Pkt Len Mean', FloatType()),
    StructField('Pkt Len Std', FloatType()),
    StructField('Pkt Len Var', FloatType()),
    StructField('FIN Flag Cnt', IntegerType()),
    StructField('SYN Flag Cnt', IntegerType()),
    StructField('RST Flag Cnt', IntegerType()),
    StructField('PSH Flag Cnt', IntegerType()),
    StructField('ACK Flag Cnt', IntegerType()),
    StructField('URG Flag Cnt', IntegerType()),
    StructField('CWE Flag Count', IntegerType()),
    StructField('ECE Flag Cnt', IntegerType()),
    StructField('Down/Up Ratio', FloatType()),
    StructField('Pkt Size Avg', FloatType()),
    StructField('Fwd Seg Size Avg', FloatType()),
    StructField('Bwd Seg Size Avg', FloatType()),
    StructField('Fwd Byts/b Avg', IntegerType()),
    StructField('Fwd Pkts/b Avg', IntegerType()),
    StructField('Fwd Blk Rate Avg', IntegerType()),
    StructField('Bwd Byts/b Avg', IntegerType()),
    StructField('Bwd Pkts/b Avg', IntegerType()),
    StructField('Bwd Blk Rate Avg', IntegerType()),
    StructField('Subflow Fwd Pkts', IntegerType()),
    StructField('Subflow Fwd Byts', IntegerType()),
    StructField('Subflow Bwd Pkts', IntegerType()),
    StructField('Subflow Bwd Byts', IntegerType()),
    StructField('Init Fwd Win Byts', IntegerType()),
    StructField('Init Bwd Win Byts', IntegerType()),
    StructField('Fwd Act Data Pkts', IntegerType()),
    StructField('Fwd Seg Size Min', IntegerType()),
    StructField('Active Mean', FloatType()),
    StructField('Active Std', FloatType()),
    StructField('Active Max', FloatType()),
    StructField('Active Min', FloatType()),
    StructField('Idle Mean', FloatType()),
    StructField('Idle Std', FloatType()),
    StructField('Idle Max', FloatType()),
    StructField('Idle Min', FloatType()),
    StructField('Label', StringType())
])

In [6]:
from pyspark.ml.feature import ChiSqSelector
from pyspark.ml.stat import Correlation

# RF model on the entire 6G balanced dataset

In [7]:
df = spark.read.csv("hdfs://172.31.8.146:9000/final_dataset.csv", schema=schema, header=True)

In [8]:
# Map "ddos" to 1 and "Benign" to 0 in the label column
# This is required by the random forest in PySpark
df = df.withColumn("Label", when(col("Label") == "ddos", 1).otherwise(0))

In [9]:
distinct_labels = df.select("Label").distinct().collect()

# Print the distinct values
for row in distinct_labels:
    print(row["Label"])

1
0


use the selected features from the feature engineering process

In [10]:
selected_features = ['Dst Port',
                     'Protocol',
                     'Flow Duration',
                     'TotLen Fwd Pkts',
                     'Fwd Pkt Len Max',
                     'Fwd Pkt Len Min',
                     'Fwd Pkt Len Std',
                     'Bwd Pkt Len Max',
                     'Bwd Pkt Len Min',
                     'Bwd Pkt Len Mean',
                     'Bwd Pkt Len Std',
                     'Flow IAT Mean',
                     'Flow IAT Std',
                     'Flow IAT Max',
                     'Flow IAT Min',
                     'Fwd IAT Tot',
                     'Fwd IAT Mean',
                     'Fwd IAT Std',
                     'Fwd IAT Max',
                     'Fwd IAT Min',
                     'Bwd IAT Tot',
                     'Bwd IAT Std',
                     'Bwd IAT Max',
                     'Pkt Len Min',
                     'Pkt Len Max',
                     'Pkt Len Mean',
                     'Pkt Len Std',
                     'Pkt Len Var',
                     'RST Flag Cnt',
                     'ECE Flag Cnt',
                     'Down/Up Ratio',
                     'Bwd Seg Size Avg',
                     'Subflow Fwd Byts',
                     'Init Fwd Win Byts',
                     'Fwd Act Data Pkts',
                     'Idle Mean',
                     'Idle Max',
                     'Idle Min']

In [11]:
selected_cols = [col for col in df.columns if col in selected_features]

# Select features
assembler = VectorAssembler(inputCols=selected_cols, outputCol="features")

### Perform data preprocessing using scaler
Feature scaling (also known as data normalization or standardization) is a preprocessing step that transforms the features (columns) of a dataset to have similar scales or ranges.

The purpose of feature scaling here is to ensure that all features contribute equally to our model training process, preventing features with larger scales from dominating those with smaller scales. It also helps the algorithms converge faster and may improve the performance of certain machine learning algorithms, particularly those that rely on distance metrics or gradient descent optimization.

In [12]:
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures")

### Use PCA for feature selection to reduce the number of features

In [13]:
# reduce to 10 principal components
pca = PCA(k=10, inputCol="scaledFeatures", outputCol="pcaFeatures")

### We adopt the hyperparameter combination of the one we got from the previous tunning processing on the small-scale dataset

## Results of using the folloing hyperparameter values
- num_trees = 10
- max_depth = 5

In [38]:
# Define fixed values for numTrees and maxDepth
num_trees = 10
max_depth = 5

### Define a new Random Forest model

In [39]:
rf = RandomForestClassifier(labelCol="Label", featuresCol="pcaFeatures", numTrees=num_trees, maxDepth=max_depth)
pipeline = Pipeline(stages=[assembler, scaler, pca, rf])

### Try k-fold cross validation (10-fold)

In [40]:
# use ROC index for evaluation
evaluator = BinaryClassificationEvaluator(labelCol="Label", metricName="areaUnderROC")
# we do not need grid search this time, but the empty parameter is required by the CrossValidator
params_grid = ParamGridBuilder().build()
# 10-fold cross-validation
crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=params_grid,
                          evaluator=evaluator,
                          numFolds=10)

In [41]:
distinct_labels = df.select("Label").distinct().collect()

# Print the distinct values
for row in distinct_labels:
    print(row["Label"])

1
0


In [42]:
# Split the data into training and test sets
(df_train, df_test) = df.randomSplit([0.7, 0.3])

In [43]:
%%time
cvModel = crossval.fit(df_train)

CPU times: user 3.39 s, sys: 633 ms, total: 4.03 s
Wall time: 36min 50s


In [44]:
%%time
# Make predictions on training set
predictions_train = cvModel.transform(df_train)

CPU times: user 15.6 ms, sys: 3.47 ms, total: 19.1 ms
Wall time: 184 ms


In [45]:
%%time
# Make predictions on validation set
predictions_test = cvModel.transform(df_test)

CPU times: user 12.9 ms, sys: 4.14 ms, total: 17 ms
Wall time: 124 ms


In [46]:
# Evaluate the performance on the training set
roc_train = evaluator.evaluate(predictions_train)
roc_train

0.994551181641148

In [47]:
# Evaluate the performance on the validation set
roc_test = evaluator.evaluate(predictions_test)
roc_test

0.994633510548535

In [51]:
# more evaluations

from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Define a multiclass classification evaluator
multiclass_evaluator = MulticlassClassificationEvaluator(labelCol="Label", predictionCol="prediction")

# Calculate accuracy
accuracy = multiclass_evaluator.evaluate(predictions_test, {multiclass_evaluator.metricName: "accuracy"})

# Calculate precision
precision = multiclass_evaluator.evaluate(predictions_test, {multiclass_evaluator.metricName: "precisionByLabel"})

# Calculate recall
recall = multiclass_evaluator.evaluate(predictions_test, {multiclass_evaluator.metricName: "recallByLabel"})

# Calculate F1 score
f1_score = multiclass_evaluator.evaluate(predictions_test, {multiclass_evaluator.metricName: "f1"})

# Print the evaluation metrics
print("=== DDoS Detection Performance on the testing set ===")
print("Accuracy: ", accuracy)
print("Precision: ", precision)
print("Recall: ", recall)
print("F1 Score: ", f1_score)
print("ROC: ", roc_test)

=== DDoS Detection Performance on the testing set ===
Accuracy:  0.9745150917548979
Precision:  0.9811725607039579
Recall:  0.9670248576003914
F1 Score:  0.9745118152367145
ROC:  0.994633510548535


In [52]:
# Calculate accuracy
accuracy_train = multiclass_evaluator.evaluate(predictions_train, {multiclass_evaluator.metricName: "accuracy"})

# Calculate precision
precision_train = multiclass_evaluator.evaluate(predictions_train, {multiclass_evaluator.metricName: "precisionByLabel"})

# Calculate recall
recall_train = multiclass_evaluator.evaluate(predictions_train, {multiclass_evaluator.metricName: "recallByLabel"})

# Calculate F1 score
f1_score_train = multiclass_evaluator.evaluate(predictions_train, {multiclass_evaluator.metricName: "f1"})

# Print the evaluation metrics
print("=== DDoS Detection Performance on the training set ===")
print("Accuracy: ", accuracy_train)
print("Precision: ", precision_train)
print("Recall: ", recall_train)
print("F1 Score: ", f1_score_train)
print("ROC: ", roc_train)

=== DDoS Detection Performance on the training set ===
Accuracy:  0.9746053844175513
Precision:  0.9813680295828863
Recall:  0.9669439940843246
F1 Score:  0.9746018040369947
ROC:  0.994551181641148


## Results of using the folloing hyperparameter values
- num_trees = 30
- max_depth = 5

In [14]:
# Define fixed values for numTrees and maxDepth
num_trees = 10
max_depth = 5

In [15]:
rf = RandomForestClassifier(labelCol="Label", featuresCol="pcaFeatures", numTrees=num_trees, maxDepth=max_depth)
pipeline = Pipeline(stages=[assembler, scaler, pca, rf])

In [16]:
# use ROC index for evaluation
evaluator = BinaryClassificationEvaluator(labelCol="Label", metricName="areaUnderROC")
# we do not need grid search this time, but the empty parameter is required by the CrossValidator
params_grid = ParamGridBuilder().build()
# 10-fold cross-validation
crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=params_grid,
                          evaluator=evaluator,
                          numFolds=10)

In [17]:
# Split the data into training and test sets
(df_train, df_test) = df.randomSplit([0.7, 0.3])

In [18]:
%%time
cvModel = crossval.fit(df_train)

24/04/29 23:06:44 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


24/04/29 23:09:31 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
24/04/29 23:09:31 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK


CPU times: user 4.02 s, sys: 905 ms, total: 4.93 s
Wall time: 40min 11s


In [19]:
%%time
# Make predictions on training set
predictions_train = cvModel.transform(df_train)

CPU times: user 19.6 ms, sys: 0 ns, total: 19.6 ms
Wall time: 240 ms


In [20]:
%%time
# Make predictions on validation set
predictions_test = cvModel.transform(df_test)

CPU times: user 16 ms, sys: 1.85 ms, total: 17.9 ms
Wall time: 175 ms


In [21]:
# Evaluate the performance on the training set
roc_train = evaluator.evaluate(predictions_train)
roc_train

0.9951357269560265

In [22]:
# Evaluate the performance on the validation set
roc_test = evaluator.evaluate(predictions_test)
roc_test

0.9952070755959271

In [23]:
# more evaluations

from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Define a multiclass classification evaluator
multiclass_evaluator = MulticlassClassificationEvaluator(labelCol="Label", predictionCol="prediction")

# Calculate accuracy
accuracy = multiclass_evaluator.evaluate(predictions_test, {multiclass_evaluator.metricName: "accuracy"})

# Calculate precision
precision = multiclass_evaluator.evaluate(predictions_test, {multiclass_evaluator.metricName: "precisionByLabel"})

# Calculate recall
recall = multiclass_evaluator.evaluate(predictions_test, {multiclass_evaluator.metricName: "recallByLabel"})

# Calculate F1 score
f1_score = multiclass_evaluator.evaluate(predictions_test, {multiclass_evaluator.metricName: "f1"})

# Print the evaluation metrics
print("=== DDoS Detection Performance on the testing set ===")
print("Accuracy: ", accuracy)
print("Precision: ", precision)
print("Recall: ", recall)
print("F1 Score: ", f1_score)
print("ROC: ", roc_test)

=== DDoS Detection Performance on the testing set ===
Accuracy:  0.9753030659027357
Precision:  0.9780745875334589
Recall:  0.9717805443255119
F1 Score:  0.9753018587169614
ROC:  0.9952070755959271


In [24]:
# Calculate accuracy
accuracy_train = multiclass_evaluator.evaluate(predictions_train, {multiclass_evaluator.metricName: "accuracy"})

# Calculate precision
precision_train = multiclass_evaluator.evaluate(predictions_train, {multiclass_evaluator.metricName: "precisionByLabel"})

# Calculate recall
recall_train = multiclass_evaluator.evaluate(predictions_train, {multiclass_evaluator.metricName: "recallByLabel"})

# Calculate F1 score
f1_score_train = multiclass_evaluator.evaluate(predictions_train, {multiclass_evaluator.metricName: "f1"})

# Print the evaluation metrics
print("=== DDoS Detection Performance on the training set ===")
print("Accuracy: ", accuracy_train)
print("Precision: ", precision_train)
print("Recall: ", recall_train)
print("F1 Score: ", f1_score_train)
print("ROC: ", roc_train)

=== DDoS Detection Performance on the training set ===
Accuracy:  0.9752463755175839
Precision:  0.9778728577190282
Recall:  0.9719040389087361
F1 Score:  0.9752452841984975
ROC:  0.9951357269560265
